# Project Web Scraping 

Books to Scrape | url of the web site : http://books.toscrape.com/index.html

In [ ]:
import csv
from bs4 import BeautifulSoup as bs
import requests

In [ ]:
try:
    url = 'http://books.toscrape.com/index.html'
    response = requests.get(url)
except:
    print(f"this url not working {response}")

In [ ]:
soup = bs(response.text,"html.parser")

In [ ]:
url = 'http://books.toscrape.com/index.html'
response = requests.get(url)

soup = bs(response.text,"html.parser")

ul = soup.find('ul', class_="nav nav-list")
href_category = ul.find_all('a')
stars_rate = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
l = []


print('-----------------------------------------------'.center(100))
print('working now to scrapte Data ? be patient'.center(100))
Co = 0
for x in href_category[1:]:
    
    Category = x.string
    Category = Category.replace("\n","").strip()
    href = x["href"]
    hreflink = 'http://books.toscrape.com/'+ href
    print(hreflink)
    for j in range(1,9):
        if j == 1:
            pass
        if j == 2:
            page_number = f"page-{j}.html" 
            hreflink = hreflink.replace("index.html",page_number)
            print(hreflink)
        if j > 2:
            page_number = f"page-{j}.html"
            hreflink = hreflink.replace(f"page-{j-1}.html",page_number)
        response = requests.get(hreflink)
        if response.status_code == 200:
            print(f">scrape link : {hreflink}")
            soup = bs(response.text, "html.parser")
            section = soup.find('section')
            list_ordonnee = section.find('ol')
            article = list_ordonnee.find_all('article',  attrs = {"class":'product_pod'})
    
            for i in article:
                Data ={}
                h3= i.find('h3')
                a = h3.find('a')
                href = a.attrs['href']
                href = "http://books.toscrape.com/catalogue"+ href[8:]
                response = requests.get(href)
                print(f">>scrape book : {href}")
                soupart = bs(response.text, "html.parser") 
                title = soupart.find('h1')
            
                Data["Genre"] = Category
                Data["Title"] = title.string
                price = soupart.find('p',attrs={'class': 'price_color'})
                price = price.text
                Data["price"] = price[2:]
                instock = soupart.find("p",class_="instock availability")
                instock = instock.text.strip()
                Data["Stock"] = instock[10:12]
                stars = soupart.find("p",class_="star-rating")
                stars = stars['class']
                stars = stars[1]
                Data["Star"] = stars_rate[stars]
                
                Table = soupart.find("table")
                UPC = Table.find_all('td')[0]
                UPC = UPC.string
                Data["UPC"] = UPC
                Type = Table.find_all('td')[1]
                Data["Type"] = Type.text
                Tax = Table.find_all('td')[4]
                Tax = Tax.text
                Data["Tax"] = Tax[2:]
                Number_of_reviews = Table.find_all('td')[-1]
                Data["Number of reviews"] = Number_of_reviews.text
                des = soupart.find_all('p')
                des = des[3]
                des = des.text
                des = " ".join(des.split(",")[:3])+"||Fore More You should read this books. Thanks"
                Data["desc"] = des
                l.append(Data)
        else:
            print(f"not found : ?? \n{hreflink}")
            


In [ ]:
with open("Books_scrapingV3.csv","w+",newline="", encoding="UTF-8") as f:
    thewriter = csv.DictWriter(f,delimiter=",",fieldnames=l[0].keys())
    thewriter.writeheader()
    for x in l:
        thewriter.writerow(x)

In [ ]:
#TODO: PRv